<a href="https://colab.research.google.com/github/VindhyaSRajan/Guided-Research/blob/master/LSTM_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
! pip install gensim
! pip install seaborn
! pip install elasticsearch
! pip install esengine
! pip install ipdb

     |████████████████████████████████| 92kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.10.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0


  Created wheel for ipdb: filename=ipdb-0.12.3-cp36-none-any.whl size=9223 sha256=a24c1a3e7418c3e05b136d771bb59a071040b58615902d10012af024a4d80efc
  Stored in directory: /root/.cache/pip/wheels/57/43/c5/614153606de8f5e358e266723f53254e70752f4ffc8c85ec63
Successfully built ipdb


In [4]:
%pwd

'/content'

In [0]:
from preprocess import *
from sub_find import *
from true_subs import *
from constants import *
# from graph_evaluation import *
from Vocabulary import Vocab
import pickle

In [0]:
from recipe import Recipe
def load_data_from_file(filename=None, ing_file_name=None, local=True, save=False):
    return load_data(file_name=filename, ing_file_name=ing_file_name, local=local, save=save)

In [7]:
import os
#os.chdir('raw_data')
recipes = load_data_from_file("100ksample", ing_file_name = "dbpedia_ingredients.txt",local=True, save=False )
clean_recipes = [x.proccessed_ing_list for x in recipes]
recipes = load_data_from_file("100ksample2", "dbpedia_ingredients.txt")
sample100k2 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample100k2) 
type(clean_recipes)

list

In [9]:
# currently looking only on the second set of elements
#TO-DO : Currently doing it for 100k samples, will later move this to 1M recipes
total_number_of_recipes = len(recipes)
print(total_number_of_recipes)

100000


In [0]:
titles_present = set()
for reduced_recipe_object in recipes:
  titles_present.add(str(reduced_recipe_object.id))

In [11]:
vars(recipes[0])

{'id': 'Cajun Spice Blend',
 'ingredients_text': '1 tablespoon garlic powder 1 tablespoon onion powder 1 tablespoon sugar 1 teaspoon salt 1 teaspoon pepper 1 teaspoon dried basil 1 teaspoon paprika 12 teaspoon cayenne pepper',
 'instructions_text': 'mix together and store in airtight container and use as wanted.',
 'proccessed_ing_list': ['garlic powder',
  'onion powder',
  'sugar',
  'cayenne pepper'],
 'title': 'Cajun Spice Blend'}

In [12]:
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
final_dataset[0]

(['garlic powder', 'onion powder', 'sugar', 'cayenne pepper'],
 'Cajun Spice Blend')

In [0]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# type(titles_present)

In [0]:
def stem_title(word):
    return stemmer.stem(word)

In [15]:
# an example
stem_title('Carsons Chicken Wellington W/ Exotic Mushroom Sherry Sauce')

'carsons chicken wellington w/ exotic mushroom sherry sauc'

In [0]:
updated_title = [stem_title(title) for title in titles_present]

In [0]:
with open('recipe_titles_stemmed.txt', 'w') as file:
     for item in updated_title:
         file.write("%s\n" % item)

In [0]:
my_list = 'cake,pasta,custard,rolls,stew,casserole,sauce,soup,burgers,pizza,salad,muffins,steaks,fish,ham'.split(",") # Do Not Run

In [0]:
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Chicken', 'Custard', 'Chips',
                'Casserole', 'Pasta', 'Pizza', 'Omelette', 'soup', 'sauce'] #Do Not Run

In [0]:
vindhya_map = {
    'Soup' : ['Soup'],
    'Sandwich' : ['Sandwich'],
    'Cocktail' : ['Cocktail'],
    'Pudding' : ['Pudding'],
    'Wraps' : ['Wraps'],
    'Doughnut' : ['Doughnut'],
    'Bread' : ['Bread'],
    'Smoothie' : ['Smoothie'],
    'Sorbet' : ['Sorbet'],
    'Stew' : ["Daal", "Chili", "Goulash", "Stew"],
    'Cake' :["Muffin", "Cake"],
    'Condiments' : ["Pickles", "Sauce"],
    'Taco' : ["Quessidila", "Fajitas", "Taco"],
    'Salad' : ["Guacamole", "Salad"],
    'Noodles' : ["Noodles"],
    'Hummus' : ['Hummus'],
    'Burger' : ['Burger'],
    'Steak' : ['Steak'],
    'Pie' : ['Pie'],
    'Custard' : ['Custard'],
    'Chips' : ['Chips'],
    'Casserole' : ['Casserole'],
    'Pasta' : ['Pasta'],
    'Pizza' : ['Pizza'],
    'Omelette' : ['Omelette'],
    'Chicken' : ['Chicken'],
    'Beef' : ['Beef'],
    'Spring Roll' : ['Spring Roll'],
    'Pancake' : ['Pancake'],
    'Pastry' : ['Pastry'],
    'Salmon' : ['Salmon'],
    'Spread' : ['Spread']
    
}

In [0]:
vindhya_map = dict((stemmer.stem(k), list(map(stemmer.stem, v))) for k, v in vindhya_map.items())

In [0]:
import itertools
all_recipe_list = list(itertools.chain(*vindhya_map.values()))

In [95]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
list(itertools.chain(*vindhya_map.values()))
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in all_recipe_list)
    
counter

43586

The next 2 cells need not to be run now. The version is updated with the grouped categories.

In [0]:
#Updated the list now. Run this cell only
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Custard', 'Chips', 'Casserole', 'Pasta', 'Pizza', 'Omelette', 'Soup', 'Sauce','Sandwich','Muffin','Pickles','Cocktail','Pudding','Wraps','Taco','Daal','Guacamole','Doughnut','Bread','Smoothie','Sorbet','Chili','Stew','Chicken','Beef',]

In [0]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in vindhya_list)
    
counter

41942

In [22]:
percentage_covered = counter / total_number_of_recipes * 100
print(percentage_covered)

43.586000000000006


In [0]:
# so sample 100k2 as we are working on the clean_recipes
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
only_ingredients = [i[0] for i in final_dataset]

In [62]:
type(final_dataset[0]) #should be a tuple

tuple

In [0]:
ingredient_vocabulary = Vocab()
for recipe in only_ingredients:
    for ingredients in recipe:
        ingredient_vocabulary.add_word(ingredients)

In [64]:
# Let us see the vocabulary size once
ingredient_vocabulary.max_idx

1152

In [65]:
ingredient_vocabulary.get_word(1151)

'crocus'

In [0]:
import pandas as pd
df = pd.DataFrame(final_dataset, columns =['Ingredients', 'Recipe_id'])

In [67]:
df[df.Recipe_id == 'Peanut Butter Pie']
df.shape

(100000, 2)

In [0]:
#Merging the two pandas df to create the one for just 100 samples per category
df2  = pd.read_csv('/content/Food_Categories_100PerCat.csv')
df2.head()
#df[df.Recipe_id == 'Screwdriver Cocktail'] #Just a sanity check that we have all the recipes in both the data frames to join by
df2['Recipe_id'] = df2.Recipe_Title.values
df2.head()
df = df.merge(df2,on = 'Recipe_id')
print(df.shape)
print(df.columns)


(3278, 4)
Index(['Ingredients', 'Recipe_id', 'Recipe_Title', 'Food_Category'], dtype='object')


In [0]:
#Sanity Check to see if all recipe categories are avaialable
print(df.shape)
df['COUNTER'] = 1
grouped_data = df.groupby(['Food_Category'])['COUNTER'].sum()
grouped_data

(3278, 4)


Food_Category
Beef         102
Bread        112
Burger       105
Cake         108
Casserole    113
Chicken      106
Chili        103
Chips        114
Cocktail     106
Custard      104
Daal           4
Doughnuts     71
Guacamole    127
Hummus       118
Muffins      105
Noodles      101
Omelette      72
Pasta        100
Pickle       109
Pie          115
Pizza        114
Pudding      108
Salad        106
Sandwich     100
Sauce        109
Smoothie     108
Sorbet       108
Soup         107
Steak        103
Stew         106
Taco         105
Wrap         109
Name: COUNTER, dtype: int64

In [0]:
#Since we have all food categories, let's drop the unecessary columns and have one final df
df = df.drop(columns = ['Recipe_Title', 'Food_Category','COUNTER'])
print(df.columns)
print(df.shape)
print('----After Dropping unwanted columns----')
print(df.columns)
print(df.shape)
df.head()

Index(['Ingredients', 'Recipe_id'], dtype='object')
(3278, 2)
----After Dropping unwanted columns----
Index(['Ingredients', 'Recipe_id'], dtype='object')
(3278, 2)


,Ingredients,Recipe_id
0,"[cream cheese, condensed milk, peanut butter, ...",Peanut Butter Pie
1,"[chocolate, sugar, butter, cream cheese, peanu...",Peanut Butter Pie
2,"[graham cracker, peanut butter, butter, cream ...",Peanut Butter Pie
3,"[graham cracker crust, cream cheese, peanut bu...",Peanut Butter Pie
4,"[cream cheese, peanut butter, sugar, milk, hea...",Peanut Butter Pie


In [0]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        result_list.append(ingredient_vocabulary.get_idx(ingedient))
    return result_list

In [0]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [98]:
df.iloc[5]

Ingredients           [orange, clove, apple, water, cinnamon]
Recipe_id                            The Spiced Cider Project
Ingredient_Numeric                       [27, 28, 29, 24, 30]
Recipe_id_numeric                                           0
Name: 5, dtype: object

In [99]:
%pwd

'/content'

In [38]:
!unzip '/content/glove.6B.100d.txt.zip'

Archive:  /content/glove.6B.100d.txt.zip
  inflating: glove.6B.100d.txt       


In [39]:
from gensim.test.utils import datapath
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/glove.6B.100d.txt')
with open('glove.100d.w2v', mode='wb+') as dest_file:
    glove2word2vec(glove_file, dest_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [40]:
import gensim
pretrained_embedding = gensim.models.KeyedVectors.load_word2vec_format('/content/glove.100d.w2v')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [100]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['salmon'].index 

7411

In [0]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        # If multi word ingredient, chose the first word
        ingedient = ingedient.split(" ")[0]
        try:
            result_list.append(pretrained_embedding.vocab[ingedient].index)
        except KeyError:
            result_list.append(pretrained_embedding.vocab['unk'].index)
    return result_list

In [0]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [103]:
df.iloc[5]

Ingredients           [orange, clove, apple, water, cinnamon]
Recipe_id                            The Spiced Cider Project
Ingredient_Numeric            [3200, 33170, 3292, 430, 17773]
Recipe_id_numeric                                           0
Name: 5, dtype: object

In [0]:
recipe_vocabulary = Vocab()
for recipe in vindhya_map:
    recipe_vocabulary.add_word(stemmer.stem(recipe))

In [105]:
recipe_vocabulary

{'<UNK>': 0, 'soup': 1, 'sandwich': 2, 'cocktail': 3, 'pud': 4, 'wrap': 5, 'doughnut': 6, 'bread': 7, 'smoothi': 8, 'sorbet': 9, 'stew': 10, 'cake': 11, 'condiment': 12, 'taco': 13, 'salad': 14, 'noodl': 15, 'hummu': 16, 'burger': 17, 'steak': 18, 'pie': 19, 'custard': 20, 'chip': 21, 'casserol': 22, 'pasta': 23, 'pizza': 24, 'omelett': 25, 'chicken': 26, 'beef': 27, 'spring rol': 28, 'pancak': 29, 'pastri': 30, 'salmon': 31, 'spread': 32}

In [106]:
len(recipe_vocabulary.word_2_idx)

33

In [107]:
recipe_vocabulary.check_any_word_in_vocab(stemmer.stem('The Spiced Cake Project'))
# recipe_vocabulary.get_idx('cake')

(True, 'cake')

In [0]:
# Uptil this point we were simply returning the first match in the recipe vocabulary. 
# This can be modified and we look for all the matches of possible recipe
# Since the list is sorted specific to our rule, we would go ahead and take the minimum value from all
def get_recipe_idx(recipe):
    recipe_stemmed = stemmer.stem(recipe)
    lookup_result = recipe_vocabulary.get_all_words_in_vocab(recipe_stemmed)
    #lookup_result = Vocab.get_all_words_in_vocab(recipe_stemmed)
    if lookup_result[0]:
        print(lookup_result[1])
        return min(lookup_result[1])
    # No match so 0 returned
    return  0

In [0]:
# def get_recipe_idx(recipe):
#     recipe_stemmed = stemmer.stem(recipe)
#     lookup_result = recipe_vocabulary.check_any_word_in_vocab(recipe_stemmed)
#     if lookup_result[0]:
#         return recipe_vocabulary.get_idx(lookup_result[1])
#     return 0

In [117]:
get_recipe_idx('The Spiced Cake Project')
#get_recipe_idx('Sugarless Applesauce Cake')

[11]


11

In [0]:
df['Recipe_id_numeric'] = df.Recipe_id.apply(get_recipe_idx)

In [115]:
df.iloc[54]

Ingredients           [seed, fruit, water, flour, baking soda, nutme...
Recipe_id                                     Sugarless Applesauce Cake
Ingredient_Numeric    [3404, 4138, 430, 8212, 10057, 30593, 17773, 6...
Recipe_id_numeric                                                     0
Name: 54, dtype: object

In [118]:
#Just Checking here that all categories have some data in them
df['COUNTER'] = 1
group_data = df.groupby(['Recipe_id_numeric'])['COUNTER'].sum()
group_data

Recipe_id_numeric
0     67055
1      3468
2      1097
3       309
4       605
5       234
6        62
7      2325
8       405
9       115
10      788
11      950
13      487
14     5729
15      218
16       94
17       84
18      797
19      791
20      191
21      817
22     1485
23     1260
24      992
25       40
26     6552
27     1448
29      416
30       69
31      842
32      275
Name: COUNTER, dtype: int64

In [127]:
df = df.drop(columns= ['COUNTER'])
df.columns

Index(['Ingredients', 'Recipe_id', 'Ingredient_Numeric', 'Recipe_id_numeric'], dtype='object')

In [0]:
df.Recipe_id_numeric.value_counts()

In [128]:
len(df)

100000

In [0]:
with open('vocab_clubbed_33.pkl','wb') as file:
    pickle.dump(recipe_vocabulary, file)
    pickle.dump(ingredient_vocabulary, file)

In [0]:
def check_empty(x):
    return len(x) == 0

In [0]:
processed_df = df.drop(df[df.Ingredient_Numeric.apply(check_empty)].index)

In [0]:
assert sum(processed_df.Ingredient_Numeric.apply(check_empty)) == 0

In [0]:
# I am just saving the file here for reference later
pd.to_pickle(processed_df, './processed_df_clubbed_33.pkl')

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

In [0]:

# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [0]:

# Some constants
batch_size = 5

In [0]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths

In [0]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y

In [0]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [0]:
df.Recipe_id_numeric[4]

In [0]:
train_dataset[0]

In [0]:
for (X,X_len,y) in train_data_loader:
    print(X)
    print(X_len)
    print(y)
    break

In [0]:
X

In [0]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim, batch_size, output_dim):
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn(1, self.batch_size, self.hidden_dim)
        self.c_n = torch.randn(1, self.batch_size, self.hidden_dim)
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted

In [0]:
vocab_size = ingredient_vocabulary.max_idx + 1

In [0]:
num_recipes = recipe_vocabulary.max_idx+1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes)

In [0]:
model(X, X_len)

In [0]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
num_epoch = 2

In [0]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
model.to(device)

In [0]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [0]:

# !pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [0]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

In [0]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [0]:
torch.__version__

In [0]:
vindhya_data_df = pd.read_csv('./recipe_title_foodCategory.csv')

In [0]:
list(set(vindhya_data_df.Food_Category))

In [0]:
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
c = ['dil','me','dard','e','disco']
sallu_bhoi = zip(a,b,c)

In [0]:
a,b,c = zip(*sallu_bhoi)

In [0]:
np.asarray(a)